# This Notebook

Congregates all previous ones into modules, and acts as an all-in-one platform to produce and interpret data from the new $n_{\rm ion}$ and $f_{\rm esc}$ project.

In [6]:
datapath = '/freya/ptmp/mpa/wuze/data/240603_test'

%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pickle

def plot_prettier(dpi=200, fontsize=10, usetex=False):
    '''
    Make plots look nicer compared to Matplotlib defaults
    Parameters: 
        dpi - int, "dots per inch" - controls resolution of PNG images that are produced
                by Matplotlib
        fontsize - int, font size to use overall
        usetex - bool, whether to use LaTeX to render fonds of axes labels 
                use False if you don't have LaTeX installed on your system
    '''
    plt.rcParams['figure.dpi']= dpi
    plt.rc("savefig", dpi=dpi)
    plt.rc('font', size=fontsize)
    plt.rc('xtick', direction='in') 
    plt.rc('ytick', direction='in')
    plt.rc('xtick.major', pad=5) 
    plt.rc('xtick.minor', pad=5)
    plt.rc('ytick.major', pad=5) 
    plt.rc('ytick.minor', pad=5)
    plt.rc('lines', dotted_pattern = [2., 2.])
    # if you don't have LaTeX installed on your laptop and this statement 
    # generates error, comment it out
    plt.rc('text', usetex=usetex)

mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('~/codes/def.mplstyle')
plot_prettier(usetex=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1D Shock Tube

In [7]:
def plot_data(fname=f'{datapath}/1d_sod_tube/Sod.block0.out1.00025.tab', x='x1v', y='rho'):
    data = np.loadtxt(fname).T
    keys = list('i x1v rho press vel1 vel2 vel3'.split())
    dataf = {keys[i]: data[i] for i in range(len(keys))}
    
    plt.plot(dataf[x], dataf[y])
    plt.xlabel(x); plt.ylabel(y)
    plt.show()

In [8]:
plot_data(fname=f'{datapath}/1d_sod_tube/Sod.block0.out1.00025.tab', x='x1v', y='rho')

FileNotFoundError: /freya/ptmp/mpa/wuze/data/240603_test/1d_sod_tube/Sod.block0.out1.00025.tab not found.

# 2D Orszag Tang Vortex

In [9]:
import h5py

ModuleNotFoundError: No module named 'h5py'

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('/Volumes/DATA/MP_multiphase_gas/athena/vis/python'))
from athena_read import athdf

In [ ]:
def get_data2d(fname=f'{datapath}/2d_orszag_tang_vortex/OrszagTang.out2.00100.athdf',
               key='rho', verbose=False):
    data = athdf(filename=fname,)
    if verbose: print(list(data.keys()))
    return data[key][0] if len(data[key]) == 1 else data[key]
    

def plot_data2d(fname=f'{datapath}/2d_orszag_tang_vortex/OrszagTang.out2.00100.athdf',
                key='rho'):
    plt.imshow(get_data2d(key=key, verbose=False), interpolation='none')
    plt.colorbar()
    plt.title(key)
    plt.show()

In [ ]:
_ = get_data2d(verbose=True)

In [ ]:
plot_data2d(key='rho')

In [ ]:
vel_img = np.array([get_data2d(key='vel1', verbose=False),
                    get_data2d(key='vel2', verbose=False),
                    get_data2d(key='vel3', verbose=False)])
vel_img = np.swapaxes(vel_img, 0, 2)

fig, ax = plt.subplots()
cax = ax.imshow(vel_img, interpolation='none')
# cbar = fig.colorbar(cax, ax=ax, orientation='vertical')
plt.title('velocity field')
plt.show()

# Turbulence simulations

## Primary quantities

In [ ]:
datapath = '/Volumes/DATA/MP_multiphase_gas'
fname = f'{datapath}/turb_240528/Turb.hst'

with open(fname, 'r') as file: keys_raw = file.readlines()[1]
keys = [a.split('=')[1] for a in keys_raw.split()[1:]]
data = np.loadtxt(fname).T
dataf = {keys[i]: data[i] for i in range(len(keys))}
print(keys)

time = dataf['time']
e_k = dataf['1-KE'] + dataf['2-KE'] + dataf['3-KE']

# plot the time evolution
plt.subplots(figsize=(4, 3))
plt.plot(time, e_k)
plt.xlabel('time'); plt.ylabel('Kinetic Energy')
plt.show()

In [ ]:
v_turb = np.sqrt(2 * e_k / dataf['mass'])
plt.subplots(figsize=(4, 3))
plt.plot(time, v_turb)
plt.axhline(1.5, alpha=0.5, lw=1, ls='--')
plt.xlabel('time'); plt.ylabel('Turbulence Velocity')
plt.show()

## Conserved quantities

In [ ]:
density = get_data2d(fname=f'{datapath}/turb_240528/Turb.out3.00004.athdf', verbose=True, key='dens')
plt.imshow(density[0], interpolation='none')
plt.title('Density')
plt.colorbar()
plt.show()

In [ ]:
# average density of the box
rho = np.mean(density)
rho

# Calculate Mach number

In [ ]:
"""Define constants"""

class unit():
    def __init__(self):        
        # length, time, and mass constants
        self.CONST_pc  = 3.086e18
        self.CONST_yr  = 3.154e7
        self.CONST_amu = 1.66053886e-24
        self.CONST_kB  = 1.3806505e-16
        self.unit_length = self.CONST_pc*1e3  # 1 kpc
        self.unit_time   = self.CONST_yr*1e6  # 1 Myr
        self.unit_density = self.CONST_amu    # 1 mp/cm-3
        self.unit_velocity = self.unit_length/self.unit_time
        self.KELVIN = self.unit_velocity*self.unit_velocity*self.CONST_amu/self.CONST_kB
        self.unit_q = (self.unit_density * (self.unit_velocity**3))/self.unit_length
        self.g = 5/3
        
        # avg atomic mass
        Xsol = 1.0
        Zsol = 1.0
        
        X = Xsol * 0.7381
        Z = Zsol * 0.0134
        Y = 1 - X - Z
        
        self.mu  = 1.0/(2.*X+ 3.*(1.-X-Z)/4.+ Z/2.);
        self.mue = 2.0/(1.0+X);
        self.muH = 1.0/X;
        self.mH = 1.0

        # alpha values for different sims
        self.alpha_hyd = 2 ** (1 / 3)  # 1.26 # 1.383
        self.alpha_mhd = (2 * 4.2 / 0.77) ** (1 / 3)

u = unit()
u.mu

In [ ]:
def calc_T(P, rho):
    """
    Calculates temeprature from constants
    ----------
    P: gas pressure
    rho: gas density
    """
    T = P/rho * u.KELVIN * u.mu
    return T

def calc_cs(T):
    """
    Calculates sound speed
    ----------
    T: temperature
    mu: avg atomic number of the gas
    """
    # convert to cm
    m_to_cm = 100

    # return np.sqrt(g.g*R*T_hot/M) * m_to_cm/g.unit_velocity
    # return np.sqrt(u.g * u.CONST_kB / (u.mu * u.CONST_amu) * T) * m_to_cm / u.unit_velocity
    return np.sqrt(u.g * u.CONST_kB / (u.mu * u.CONST_amu) * T) / u.unit_velocity

def calc_mach(v_turb, P, rho):
    """
    Calculates the Mach number
    ----------
    v_turb: turbulence velocity
    P, rho
    """
    T = calc_T(P, rho)
    cs = calc_cs(T)
    print(f'cs = {cs}')
    return v_turb / cs

# def calc_dedt_mach(mach, P, rho, L):
#     """
#     Returns required dedt for a given Mach number, density, temperature, and box size
#     """
#     # calculate sound speed first
#     T = calc_T(P, rho)
#     cs_new = calc_cs(T)
#     print(f"cs_hot: {cs_new}")

#     dedt_req = rho * (cs_new**3) * (L**2) * (mach**3) / (u.alpha_hyd**3)
#     return dedt_req


def calc_dedt_mach(v_turb, rho, L):
    """
    Returns required dedt for a given Mach number, density, temperature, and box size

    An updated version of the function that uses turbulent velocity directly
    Does NOT calculate sound speed, so no pressure required
    """
    
    dedt_req = rho * v_turb**3 * (L**2) / (u.alpha_hyd**3)
    return dedt_req

In [ ]:
# calculate t_eddie
t_eddie = 1/v_turb
# approximate t_corr
t_corr = np.mean(t_eddie)
print(t_corr)

# apprximate dtdrive = 0.01 * t_eddie
dtdrive = 0.01 * np.mean(t_eddie)
print(dtdrive)

## Predict dedt

In [ ]:
# calculate debt needed to achieve 0.5 mach
v_turb_end = 2
calc_dedt_mach(v_turb_end,
               rho=rho,
               L=1)  # box size = x1max - x1min

In [ ]:
"""Check for convergence"""

datapath = '/Volumes/DATA/MP_multiphase_gas'
fname = f'{datapath}/turb_240528/Turb.hst'

with open(fname, 'r') as file: keys_raw = file.readlines()[1]
keys = [a.split('=')[1] for a in keys_raw.split()[1:]]
data = np.loadtxt(fname).T
dataf = {keys[i]: data[i] for i in range(len(keys))}
print(keys)

time = dataf['time']
e_k = dataf['1-KE'] + dataf['2-KE'] + dataf['3-KE']

v_turb = np.sqrt(2 * e_k / dataf['mass'])
plt.subplots(figsize=(4, 3))
plt.plot(time, v_turb)
plt.axhline(v_turb_end, alpha=0.5, lw=1, ls='--')
plt.xlabel('time'); plt.ylabel('Turbulence Velocity')
plt.show()

# Turbulent box analysis

In [ ]:
_ = get_data2d(fname=f'{datapath}/turb_240529/turb/Turb.out2.00100.athdf', verbose=True, key='press')

## Turbulence

In [ ]:
"""Check for convergence"""

datapath = '/Volumes/DATA/MP_multiphase_gas'
fname = f'{datapath}/turb_240529/turb/Turb.hst'

with open(fname, 'r') as file: keys_raw = file.readlines()[1]
keys = [a.split('=')[1] for a in keys_raw.split()[1:]]
data = np.loadtxt(fname).T
dataf = {keys[i]: data[i] for i in range(len(keys))}
print(keys)

time = dataf['time']
e_k = dataf['1-KE'] + dataf['2-KE'] + dataf['3-KE']

v_turb = np.sqrt(2 * e_k / dataf['mass'])
plt.subplots(figsize=(4, 3))
plt.plot(time, v_turb)
# plt.axhline(v_turb_end, alpha=0.5, lw=1, ls='--')
plt.xlabel('time'); plt.ylabel('Turbulence Velocity')
plt.show()

In [ ]:
# def visualize_3d(fname=f'{datapath}/turb_240528/Turb.out3.00004.athdf', verbose=True, key='dens'):
fname=f'{datapath}/turb_240529/cloud/Turb.out2.00101.athdf'; verbose=False
%matplotlib widget
from matplotlib.widgets import Button, Slider

# plot density
z_range = [0, 31]
z_init = 10
mom_comp = [get_data2d(fname=fname, verbose=verbose, key='vel1') * 5,
            get_data2d(fname=fname, verbose=verbose, key='vel2') * 5,
            get_data2d(fname=fname, verbose=verbose, key='vel3') * 5]
mom_comp = np.array(mom_comp)

density = np.swapaxes(mom_comp, 0, 3)
fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
img = plt.imshow(density[z_init], interpolation='none')
# plt.title('Density')
plt.colorbar()

plt.ioff()

# Make a horizontal slider to control the z slice
ax_slider = fig.add_axes([0.25, 0.90, 0.5, 0.05])
redshift_slider = Slider(
    ax=ax_slider,
    label=r'z value',
    valmin=z_range[0],
    valmax=z_range[1],
    valinit=z_init,
    valstep=1.
)

# The function to be called anytime a slider's value changes
def update(z):
    # update to redshift
    img.set_data(density[int(z)])
    fig.canvas.draw_idle()
redshift_slider.on_changed(update)  # register the update function with each slider

# Create a button to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.9, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')
def reset(event):
    redshift_slider.reset()
button.on_clicked(reset)

## Mach number

In [ ]:
# change back to inline plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('/Users/jasonwu/Programs/codes/def.mplstyle')
plot_prettier(usetex=True)

In [ ]:
datapath = '/Volumes/DATA/MP_multiphase_gas'
fname = f'{datapath}/turb_240531_2/turb/Turb.hst'

# calculate v_turb
with open(fname, 'r') as file: keys_raw = file.readlines()[1]
keys = [a.split('=')[1] for a in keys_raw.split()[1:]]
data = np.loadtxt(fname).T
dataf = {keys[i]: data[i] for i in range(len(keys))}
print(keys)
time = dataf['time']
e_k = dataf['1-KE'] + dataf['2-KE'] + dataf['3-KE']
v_turb = np.sqrt(2 * e_k / dataf['mass'])

# calculate mach number
fname=f'{datapath}/turb_240531_2/turb/Turb.out2.00100.athdf'
rho = get_data2d(fname=fname, verbose=False, key='rho')
press = get_data2d(fname=fname, verbose=False, key='press')
T_scale = 4e6
mach = v_turb / calc_cs(T_scale)

# mach number from the turb.hst file
mach_sim = v_turb / dataf['c_s_sum'] * 32**3

plt.subplots(figsize=(4, 3))
plt.plot(time, mach, label='Calculated')
plt.plot(time, mach_sim, label='hst file')
plt.xlabel('time'); plt.ylabel('Mach number')
plt.legend()
plt.show()

## Temperature

### Spatial visualizations

In [ ]:
# def visualize_3d(fname=f'{datapath}/turb_240528/Turb.out3.00004.athdf', verbose=True, key='dens'):
fname=f'{datapath}/turb_240531_2/cloud/Turb.out2.00151.athdf'; verbose=True
%matplotlib widget
from matplotlib.widgets import Button, Slider

# plot density
z_range = [0, 31]
z_init = 15
rho = get_data2d(fname=fname, verbose=verbose, key='rho')
press = get_data2d(fname=fname, verbose=verbose, key='press')
temperature = calc_T(press, rho)
fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
img = plt.imshow(temperature[z_init], interpolation='none', norm='log', vmin=400, vmax=100000000)
# plt.title('Density')
plt.colorbar()

plt.ioff()

# Make a horizontal slider to control the z slice
ax_slider = fig.add_axes([0.25, 0.90, 0.5, 0.05])
redshift_slider = Slider(
    ax=ax_slider,
    label=r'z value',
    valmin=z_range[0],
    valmax=z_range[1],
    valinit=z_init,
    valstep=1.
)

# The function to be called anytime a slider's value changes
def update(z):
    # update to redshift
    img.set_data(temperature[int(z)])
    fig.canvas.draw_idle()
redshift_slider.on_changed(update)  # register the update function with each slider

# Create a button to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.9, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')
def reset(event):
    redshift_slider.reset()
button.on_clicked(reset)

In [ ]:
# def visualize_3d(fname=f'{datapath}/turb_240528/Turb.out3.00004.athdf', verbose=True, key='dens'):
fname=f'{datapath}/turb_240531_2/cloud/Turb.out2.00101.athdf'; verbose=True
%matplotlib widget
from matplotlib.widgets import Button, Slider

# plot density
z_range = [0, 31]
z_init = 15
rho = get_data2d(fname=fname, verbose=verbose, key='rho')
press = get_data2d(fname=fname, verbose=verbose, key='press')
temperature = calc_T(press, rho)
fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
img = plt.imshow(temperature[z_init], interpolation='none', norm='log', vmin=400, vmax=100000000)
# plt.title('Density')
plt.colorbar()

plt.ioff()

# Make a horizontal slider to control the z slice
ax_slider = fig.add_axes([0.25, 0.90, 0.5, 0.05])
redshift_slider = Slider(
    ax=ax_slider,
    label=r'z value',
    valmin=z_range[0],
    valmax=z_range[1],
    valinit=z_init,
    valstep=1.
)

# The function to be called anytime a slider's value changes
def update(z):
    # update to redshift
    img.set_data(temperature[int(z)])
    fig.canvas.draw_idle()
redshift_slider.on_changed(update)  # register the update function with each slider

# Create a button to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.9, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')
def reset(event):
    redshift_slider.reset()
button.on_clicked(reset)

In [ ]:
# change back to inline plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('/Users/jasonwu/Programs/codes/def.mplstyle')
plot_prettier(usetex=True)

### Hist

In [ ]:
"""Define constants for the run"""

T_scale = 4e6
cloud_chi = 5000; T_cold = T_scale / 5000

T_cut = 2e6

In [ ]:
fname=f'{datapath}/turb_240531_2/turb/Turb.out2.00100.athdf'
# plot density
z_range = [0, 31]
z_init = 15
rho = get_data2d(fname=fname, verbose=False, key='rho')
press = get_data2d(fname=fname, verbose=False, key='press')
temperature = calc_T(press, rho)

plt.subplots(figsize=(4, 3))
plt.hist(temperature.flatten(), alpha=0.5,
         bins=np.logspace(np.log10(800), np.log10(2e7), 100))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$T$ [K]')
plt.axvline(T_scale, color='red', lw=1, ls='--')
plt.axvline(T_cold, color='blue', lw=1, ls='--')
plt.text(1e4, 1000, 'Before cloud')
plt.show()

In [ ]:
fname=f'{datapath}/turb_240531_2/cloud/Turb.out2.00101.athdf'; verbose=True
# plot density
rho = get_data2d(fname=fname, verbose=False, key='rho')
press = get_data2d(fname=fname, verbose=False, key='press')
temperature = calc_T(press, rho)

plt.subplots(figsize=(4, 3))
plt.hist(temperature.flatten(), alpha=0.5,
         bins=np.logspace(np.log10(800), np.log10(2e7), 100))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$T$ [K]')
plt.axvline(T_scale, color='red', lw=1, ls='--', label='hot')
plt.axvline(T_cut, color='orange', lw=1, ls='-.', label='cooling lim')
plt.axvline(4e4, color='green', lw=1, ls='--', label='warm')
plt.axvline(T_cold, color='blue', lw=1, ls='--', label='cold')
plt.legend()
plt.show()

In [ ]:
fname=f'{datapath}/turb_240531_2/cloud/Turb.out2.00200.athdf'; verbose=True
# plot density
rho = get_data2d(fname=fname, verbose=False, key='rho')
press = get_data2d(fname=fname, verbose=False, key='press')
temperature = calc_T(press, rho)

plt.subplots(figsize=(4, 3))
plt.hist(temperature.flatten(), alpha=0.5,
         bins=np.logspace(np.log10(800), np.log10(2e7), 100))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$T$ [K]')
plt.axvline(T_scale, color='red', lw=1, ls='--', label='hot')
plt.axvline(T_cut, color='orange', lw=1, ls='-.', label='cooling lim')
plt.axvline(4e4, color='green', lw=1, ls='--', label='warm')
plt.axvline(T_cold, color='blue', lw=1, ls='--', label='cold')
plt.legend()
plt.show()

### Evolution

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

# grab the temperature
fname=f'{datapath}/turb_240531_2/cloud/Turb.out2.00101.athdf'; verbose=True
# plot density
z_range = [0, 31]
z_init = 15
rho = get_data2d(fname=fname, verbose=verbose, key='rho')
press = get_data2d(fname=fname, verbose=verbose, key='press')
temperature = calc_T(press, rho)

# visualize
X, Y, Z = np.mgrid[-16:15:32j, -16:15:32j, -16:15:32j]
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=np.log10(temperature.flatten()),
    opacity=0.1,  # needs to be small to see through all surfaces
    surface_count=1,  # Display only one surface
    isomin=np.log10(T_cold),  # Minimum temperature for isosurface
    isomax=np.log10(T_cold),  # Maximum temperature for isosurface
    colorscale=[[0.0, "rgb(0,0,0)"], [1.0, "rgb(0,0,0)"]],#'RdBu_r',
    colorbar=dict(title='Temperature'),
    # cmin=1,  # Set the minimum color scale value
    # cmax=7  # Set the maximum color scale value (adjust as needed)
), layout={'height': 800})
fig.show()

In [ ]:
# import plotly.graph_objects as go
# import numpy as np

# # construct a list of files
# file_list = []
# for fnum in np.arange(101, 201, 1).astype(int):
#     fname = f'{datapath}/turb_240531_2/cloud/Turb.out2.00{fnum:.0f}.athdf'
#     file_list.append(fname)

# # Generate grid for plotting
# X, Y, Z = np.mgrid[-16:15:32j, -16:15:32j, -16:15:32j]

# # Create initial figure
# fig = go.Figure(layout={'height': 800})

# # Initialize frames list
# frames = []

# # Loop over filenames to create frames
# for fname in file_list:
#     rho = get_data2d(fname=fname, verbose=False, key='rho')
#     press = get_data2d(fname=fname, verbose=False, key='press')
#     temperature = calc_T(press, rho)
    
#     frames.append(go.Frame(
#         data=[go.Volume(
#             x=X.flatten(),
#             y=Y.flatten(),
#             z=Z.flatten(),
#             value=np.log10(temperature.flatten()),
#             opacity=0.1,  # needs to be small to see through all surfaces
#             surface_count=1,  # Display only one surface
#             isomin=np.log10(T_cold),  # Minimum temperature for isosurface
#             isomax=np.log10(T_cold),  # Maximum temperature for isosurface
#             colorscale=[[0.0, "rgb(0,0,0)"], [1.0, "rgb(0,0,0)"]],#'RdBu_r',
#             colorbar=dict(title='Temperature'),
#             cmin=2,  # Set the minimum color scale value
#             cmax=7  # Set the maximum color scale value (adjust as needed)
#         )],
#         name=fname
#     ))

# # Add initial data
# rho = get_data2d(fname=file_list[0], verbose=False, key='rho')
# press = get_data2d(fname=file_list[0], verbose=False, key='press')
# temperature = calc_T(press, rho)

# fig.add_trace(go.Volume(
#     x=X.flatten(),
#     y=Y.flatten(),
#     z=Z.flatten(),
#     value=np.log10(temperature.flatten()),
#     opacity=0.1,  # needs to be small to see through all surfaces
#     surface_count=1,  # Display only one surface
#     isomin=np.log10(T_cold),  # Minimum temperature for isosurface
#     isomax=np.log10(T_cold),  # Maximum temperature for isosurface
#     colorscale=[[0.0, "rgb(0,0,0)"], [1.0, "rgb(0,0,0)"]],#'RdBu_r',
#     colorbar=dict(title='Temperature')
# ))

# # Add frames to figure
# fig.frames = frames

# # Add animation settings
# fig.update_layout(
#     updatemenus=[{
#         'buttons': [
#             {
#                 'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
#                 'label': 'Play',
#                 'method': 'animate'
#             },
#             {
#                 'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
#                 'label': 'Pause',
#                 'method': 'animate'
#             }
#         ],
#         'direction': 'left',
#         'pad': {'r': 10, 't': 87},
#         'showactive': False,
#         'type': 'buttons',
#         'x': 0.1,
#         'xanchor': 'right',
#         'y': 0,
#         'yanchor': 'top'
#     }]
# )

# fig.update_layout(
#     sliders=[{
#         'active': 0,
#         'yanchor': 'top',
#         'xanchor': 'left',
#         'currentvalue': {
#             'font': {'size': 20},
#             'prefix': 'Frame:',
#             'visible': True,
#             'xanchor': 'right'
#         },
#         'transition': {'duration': 30},
#         'pad': {'b': 10, 't': 50},
#         'len': 0.9,
#         'x': 0.1,
#         'y': 0,
#         'steps': [{
#             'args': [[fname], {'frame': {'duration': 30, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 30}}],
#             'label': fname,
#             'method': 'animate'
#         } for fname in file_list]
#     }]
# )

# fig.write_html("temperature_animation_fast.html")
# fig.show()

## Other quantities

In [ ]:
# def visualize_3d(fname=f'{datapath}/turb_240528/Turb.out3.00004.athdf', verbose=True, key='dens'):
fname=f'{datapath}/turb_240529/cloud/Turb.out2.00200.athdf'; verbose=True; key = 'rho'
%matplotlib widget
from matplotlib.widgets import Button, Slider

# plot density
z_range = [0, 31]
z_init = 10
data = get_data2d(fname=fname, verbose=verbose, key=key)
fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
img = plt.imshow(data[z_init], interpolation='none', norm='log')
# plt.title('Density')
plt.colorbar()

plt.ioff()

# Make a horizontal slider to control the z slice
ax_slider = fig.add_axes([0.25, 0.90, 0.5, 0.05])
redshift_slider = Slider(
    ax=ax_slider,
    label=r'z value',
    valmin=z_range[0],
    valmax=z_range[1],
    valinit=z_init,
    valstep=1.
)

# The function to be called anytime a slider's value changes
def update(z):
    # update to redshift
    img.set_data(data[int(z)])
    fig.canvas.draw_idle()
redshift_slider.on_changed(update)  # register the update function with each slider

# Create a button to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.9, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')
def reset(event):
    redshift_slider.reset()
button.on_clicked(reset)

# Check cold gas evolution